In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the data (replace with your file path or data loading procedure)
df = pd.read_csv("vote_and_gun.csv")

# 2. Basic info about data
print("=== Basic Info ===")
print(df.info())
print("\n=== First 5 Rows ===")
print(df.head())
print("\n=== Missing Values ===")
print(df.isna().sum())

# 3. Examine distributions for categorical variables

# List of categorical columns for 2016 and 2020
categorical_cols = [
    "16Vote", "16StrongVote", "16Closer", "16VoteSum",
    "16GunHarder", "16GunImportance",
    "20Vote", "20StrongVote", "20Closer", "20VoteSum",
    "20GunHarder", "20GunImportance"
]

for col in categorical_cols:
    print(f"\n=== Value Counts for {col} ===")
    print(df[col].value_counts(dropna=False))
    
    # Create bar plot
    plt.figure(figsize=(6, 4))
    df[col].value_counts(dropna=False).plot(kind='bar')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(f"distribution_of_{col}.png")
    plt.close()  # Close the plot so it doesn't interfere with the next

# 4. Examine numeric columns (gun ownership counts)
numeric_cols = ["16GunHowMany", "20GunHowMany"]

print("\n=== Descriptive Statistics for Numeric Columns ===")
print(df[numeric_cols].describe())

for col in numeric_cols:
    # Histogram
    plt.figure(figsize=(6, 4))
    df[col].dropna().hist(bins=20)
    plt.title(f"Histogram of {col}")
    plt.xlabel("Count")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(f"histogram_of_{col}.png")
    plt.close()

# 5. (Optional) Correlation among numeric columns
# If you only have these two numeric columns, the correlation matrix is small,
# but this shows how you'd do it if more numeric columns existed.
corr_matrix = df[numeric_cols].corr()
print("\n=== Correlation Matrix (Numeric Columns) ===")
print(corr_matrix)

plt.figure(figsize=(5, 4))
sns.heatmap(corr_matrix, annot=True, cmap="Blues", vmin=-1, vmax=1)
plt.title("Correlation Heatmap: Gun Ownership Counts (16 vs 20)")
plt.tight_layout()
plt.savefig("correlation_heatmap_gun_counts.png")
plt.close()

print("\nEDA script complete. Check the generated plots for distributions and patterns.")

=== Basic Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2839 entries, 0 to 2838
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   16Vote           2839 non-null   object 
 1   16StrongVote     2839 non-null   object 
 2   16Closer         2839 non-null   object 
 3   16VoteSum        2839 non-null   object 
 4   16GunHarder      2839 non-null   object 
 5   16GunImportance  2839 non-null   object 
 6   16GunHowMany     2686 non-null   float64
 7   20Vote           2839 non-null   object 
 8   20StrongVote     2839 non-null   object 
 9   20Closer         2839 non-null   object 
 10  20VoteSum        2839 non-null   object 
 11  20GunHarder      2839 non-null   object 
 12  20GunImportance  2839 non-null   object 
 13  20GunHowMany     2624 non-null   float64
dtypes: float64(2), object(12)
memory usage: 310.6+ KB
None

=== First 5 Rows ===
        16Vote 16StrongVote      16Closer               

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 1. Load the processed data
df = pd.read_csv("vote_and_gun.csv")

# 2. Create a difference in gun ownership between 2020 and 2016
#    We'll store it in a new column "gun_diff"
df["gun_diff"] = df["20GunHowMany"] - df["16GunHowMany"]

# 3. We might want to encode the 2016 or 2020 vote into numeric indicators for regression
#    For example, let's create a binary variable: 1 = Democrat, 0 = Otherwise
#    (You can expand to multiple dummies for different categories if you prefer.)
def democrat_indicator(v):
    return 1 if v == "Democrat" else 0

df["vote16_dem"] = df["16Vote"].apply(democrat_indicator)
df["vote20_dem"] = df["20Vote"].apply(democrat_indicator)

# 4. Drop rows where gun_diff or any key predictor is missing
df_ols = df.dropna(subset=["gun_diff", "vote16_dem", "vote20_dem"])

# 5. Quick descriptive check
print("Number of rows in df_ols:", len(df_ols))
print(df_ols[["16Vote", "20Vote", "gun_diff"]].head())

Number of rows in df_ols: 2543
        16Vote       20Vote  gun_diff
0   Republican   Republican      15.0
1   Republican  Independent       0.0
3  Independent  Independent       0.0
4     Democrat     Democrat       0.0
5  Independent  Independent       1.0


In [3]:
# OLS: gun_diff ~ 2016 Democrat vote + 2020 Democrat vote
model = smf.ols("gun_diff ~ vote16_dem + vote20_dem", data=df_ols)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               gun_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.506
Date:                Thu, 16 Jan 2025   Prob (F-statistic):             0.0818
Time:                        20:45:56   Log-Likelihood:                -7640.0
No. Observations:                2543   AIC:                         1.529e+04
Df Residuals:                    2540   BIC:                         1.530e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2766      0.126      2.191      0.0

In [4]:
# Example: add a measure of how strict they want gun laws in 2016
# We'll treat "16GunHarder" as a factor (categorical).
# We must transform it to numeric dummies, or use C() in the formula.

# Let's create a simpler numeric code: 
# More strict = 1, Less strict = -1, Same = 0, No answer = NaN
map_gun_harder = {"More strict":1, "Less strict":-1, "Same as now":0}
df_ols["gunHarder2016_num"] = df_ols["16GunHarder"].map(map_gun_harder)

# Now include it
model2 = smf.ols("gun_diff ~ vote16_dem + vote20_dem + gunHarder2016_num", data=df_ols)
results2 = model2.fit()
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:               gun_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.675
Date:                Thu, 16 Jan 2025   Prob (F-statistic):              0.170
Time:                        20:46:51   Log-Likelihood:                -7627.5
No. Observations:                2538   AIC:                         1.526e+04
Df Residuals:                    2534   BIC:                         1.529e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.2861      0.13

/tmp/ipykernel_156536/2063633134.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ols["gunHarder2016_num"] = df_ols["16GunHarder"].map(map_gun_harder)


In [6]:
import statsmodels.api as sm

# 1. We'll define the outcome
#    We already created vote20_dem above (1 if Dem, 0 otherwise).

# 2. Potential predictors
#    - Whether they voted Dem in 2016 (vote16_dem)
#    - 2016 gun ownership (16GunHowMany)
#    - 2016 gun stricter attitude (gunHarder2016_num)
#    - etc.

df_logit = df.dropna(subset=["vote20_dem", "vote16_dem", "16GunHowMany", "16GunHarder"])

# 3. Convert "16GunHarder" to numeric as before, or create dummy variables
df_logit["gunHarder2016_num"] = df_logit["16GunHarder"].map(map_gun_harder)
df_logit = df_logit.dropna()

# 4. Prepare X and y
X = df_logit[["vote16_dem", "16GunHowMany", "gunHarder2016_num"]]
y = df_logit["vote20_dem"]

# 5. statsmodels requires adding a constant for intercept
X = sm.add_constant(X)

# 6. Fit logistic regression
logit_model = sm.Logit(y, X)
logit_results = logit_model.fit()
print(logit_results.summary())

Optimization terminated successfully.
         Current function value: 0.381789
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             vote20_dem   No. Observations:                 2538
Model:                          Logit   Df Residuals:                     2534
Method:                           MLE   Df Model:                            3
Date:                Thu, 16 Jan 2025   Pseudo R-squ.:                  0.4238
Time:                        20:49:12   Log-Likelihood:                -968.98
converged:                       True   LL-Null:                       -1681.7
Covariance Type:            nonrobust   LLR p-value:                8.837e-309
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -2.4664      0.109    -22.622      0.000      -2.680      -2.253
vote16_d

/tmp/ipykernel_156536/3062107113.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_logit["gunHarder2016_num"] = df_logit["16GunHarder"].map(map_gun_harder)


In [9]:
df['id'] = np.arange(0, len(df))
df.head()

,16Vote,16StrongVote,16Closer,16VoteSum,16GunHarder,16GunImportance,16GunHowMany,20Vote,20StrongVote,20Closer,20VoteSum,20GunHarder,20GunImportance,20GunHowMany,gun_diff,vote16_dem,vote20_dem,id
0,Republican,Strong,Inapplicable,Strong Republican,Same as now,Important,10.0,Republican,Strong,Inapplicable,Strong Republican,Same as now,Neutral,25.0,15.0,0,0,0
1,Republican,Weak,Inapplicable,Not very strong Republican,Same as now,Most important,0.0,Independent,No answer,Neither,Independent,Same as now,Most important,0.0,0.0,0,0,1
2,No answer,No answer,Republican,Independent-Republican,Same as now,Most important,NaN,Other,No answer,Republican,Independent-Republican,Same as now,Important,1.0,NaN,0,0,2
3,Independent,No answer,Democrat,Independent-Democrat,More strict,Most important,0.0,Independent,No answer,Neither,Independent,Same as now,Neutral,0.0,0.0,0,0,3
4,Democrat,Strong,Inapplicable,Strong Democrat,More strict,Most important,0.0,Democrat,Strong,Inapplicable,Strong Democrat,More strict,Important,0.0,0.0,1,1,4


In [11]:
from linearmodels import PanelOLS
df_16 = df[["id", "16GunHowMany", "16Vote", "16GunHarder"]].copy()
df_16["Year"] = 2016
df_16.rename(
    columns={
        "16GunHowMany": "GunHowMany",
        "16Vote": "Vote",
        "16GunHarder": "GunHarder"
    },
    inplace=True
)

# 2020 subset
df_20 = df[["id", "20GunHowMany", "20Vote", "20GunHarder"]].copy()
df_20["Year"] = 2020
df_20.rename(
    columns={
        "20GunHowMany": "GunHowMany",
        "20Vote": "Vote",
        "20GunHarder": "GunHarder"
    },
    inplace=True
)

# Concatenate
df_panel = pd.concat([df_16, df_20], ignore_index=True)

# Set a MultiIndex of (RespondentID, Year)
df_panel = df_panel.set_index(["id", "Year"])

print(df_panel.head())

         GunHowMany         Vote    GunHarder
id Year                                      
0  2016        10.0   Republican  Same as now
1  2016         0.0   Republican  Same as now
2  2016         NaN    No answer  Same as now
3  2016         0.0  Independent  More strict
4  2016         0.0     Democrat  More strict


In [12]:
# Binary indicator: 1 if "Democrat", 0 otherwise
df_panel["isDem"] = (df_panel["Vote"] == "Democrat").astype(int)

In [13]:
map_gun_harder = {
    "More strict": 1,
    "Same as now": 0,
    "Less strict": -1,
    "No answer": np.nan
}

df_panel["GunHarder_num"] = df_panel["GunHarder"].map(map_gun_harder)

In [14]:
df_panel.dropna(subset=["GunHowMany", "isDem", "GunHarder_num"], inplace=True)

In [15]:
# Setup PanelOLS
model = PanelOLS(
    df_panel["GunHowMany"],                 # Dependent variable
    df_panel[["isDem", "GunHarder_num"]],   # Exogenous regressors
    entity_effects=True,    # Per-RespondentID fixed effect
    time_effects=True       # Per-Year fixed effect
)

results = model.fit()
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:             GunHowMany   R-squared:                        0.0034
Estimator:                   PanelOLS   R-squared (Between):             -0.0365
No. Observations:                5147   R-squared (Within):               0.0038
Date:                Thu, Jan 16 2025   R-squared (Overall):             -0.0304
Time:                        20:57:04   Log-likelihood                 -1.18e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4.0968
Entities:                        2755   P-value                           0.0167
Avg Obs:                       1.8682   Distribution:                  F(2,2389)
Min Obs:                       1.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             4.0968
                            

In [16]:
results = model.fit(cov_type="clustered", cluster_entity=True)

In [17]:
fe = results.estimated_effects
print(fe.head())

         estimated_effects
id Year                   
0  2016          17.393700
1  2016          -0.106300
3  2016           0.064783
4  2016           0.782677
5  2016           0.222618
